In [9]:
from __future__ import print_function
import tensorflow.keras as keras
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Flatten
from tensorflow.keras.layers import Conv2D, MaxPooling2D
from tensorflow.keras.layers import Activation

batch_size = 128
num_classes = 10
epochs = 12

# input image dimensions
img_rows, img_cols = 28, 28

# the data, split between train and test sets
(x_train, y_train), (x_test, y_test) = mnist.load_data()

x_train = x_train.reshape(x_train.shape[0], img_rows, img_cols, 1)
x_test = x_test.reshape(x_test.shape[0], img_rows, img_cols, 1)
input_shape = (img_rows, img_cols, 1)

## normalise data
x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


#model definition
model = Sequential()

model.add(Conv2D(32, (3, 3), input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(num_classes, activation='softmax'))

model.compile(loss=keras.losses.categorical_crossentropy,
              optimizer=keras.optimizers.Adadelta(),
              metrics=['accuracy'])

model.fit(x_train, y_train,
          batch_size=batch_size,
          epochs=epochs,
          verbose=1,
          validation_data=(x_test, y_test))
score = model.evaluate(x_test, y_test, verbose=0)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

x_train shape: (60000, 28, 28, 1)
60000 train samples
10000 test samples
Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Train on 60000 samples, validate on 10000 samples
Epoch 1/12
60000/60000 [==============================] - 78s 1ms/sample - loss: 2.2643 - acc: 0.2379 - val_loss: 2.2085 - val_acc: 0.4433
Epoch 2/12
60000/60000 [==============================] - 79s 1ms/sample - loss: 2.1451 - acc: 0.5511 - val_loss: 2.0621 - val_acc: 0.6362
Epoch 3/12
60000/60000 [==============================] - 78s 1ms/sample - loss: 1.9603 - acc: 0.6595 - val_loss: 1.8229 - val_acc: 0.6973
Epoch 4/12
60000/60000 [==============================] - 79s 1ms/sample - loss: 1.6685 - acc: 0.7227 - val_loss: 1.4716 - val_acc: 0.7727
Epoch 5/12
60000/60000 [==============================] - 77s 1ms/sample - loss: 1.3002 - acc: 0.7836 - val_loss: 1.0976 - val_acc: 0.8146
Epoch 6/12
60000/60000 [==============================] - 81s 1ms/sample - loss: 0.9766 - acc: 0.8116

In [10]:
## Save your model's weights for future private prediction
model.save('short-conv-mnist.h5')

Part 2: Secure Model Serving with Syft Keras
Now that you have a trained model with normal Keras, you are ready to serve some private predictions. We can do that using Syft Keras.

To secure and serve this model, we will need three TFEWorkers (servers). This is because TF Encrypted under the hood uses an encryption technique called multi-party computation (MPC). The idea is to split the model weights and input data into shares, then send a share of each value to the different servers. The key property is that if you look at the share on one server, it reveals nothing about the original value (input data or model weights).

We'll define a Syft Keras model like we did in the previous notebook. However, there is a trick: before instantiating this model, we'll run hook = sy.KerasHook(tf.keras). This will add three important new methods to the Keras Sequential class:

<ol>
    <li>
share: will secure your model via secret sharing; by default, it will use the SecureNN protocol from TF Encrypted to secret share your model between each of the three TFEWorkers. Most importantly, this will add the capability of providing predictions on encrypted data.
    </li>
    <li>
serve: this function will launch a serving queue, so that the TFEWorkers can accept prediction requests on the secured model from external clients.
    </li>
    <li>
shutdown_workers: once you are done providing private predictions, you can shut down your model by running this function. It will direct you to shutdown the server processes manually if you've opted to manually manage each worker.
If you want to learn more about MPC, you can read this excellent blog.
    </li>
</ol>

LOAD & SERVE MODEL

In [12]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import AveragePooling2D, Conv2D, Dense, Activation, Flatten, ReLU, Activation

import syft as sy
hook = sy.KerasHook(tf.keras)

In [ ]:
num_classes = 10
input_shape = (1, 28, 28, 1)

model = Sequential()

model.add(Conv2D(32, (3, 3), batch_input_shape=input_shape))
model.add(Activation('relu'))
model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D((2, 2)))
model.add(Flatten())
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dense(num_classes, name="logit"))

LOADING PRE-TRAINED WEIGHTS

In [15]:
pre_trained_weights = 'short-conv-mnist.h5'
model.load_weights(pre_trained_weights)

SETTING UP WORKERS

In [16]:
AUTO = True

alice = sy.TFEWorker(host='localhost:4000', auto_managed=AUTO)
bob = sy.TFEWorker(host='localhost:4001', auto_managed=AUTO)
carol = sy.TFEWorker(host='localhost:4002', auto_managed=AUTO)

cluster = sy.TFECluster(alice, bob, carol)
cluster.start()

if AUTO is True, there is no need to start servers.

Splitting the model into shares

In [17]:
model.share(cluster)




Instructions for updating:
ksizes is deprecated, use sizes instead
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where





In [18]:
## limiting the number of requests to 3 unless otherwise stated for testing
## to set no limit depending on requirement

model.serve(num_requests=3)

Served encrypted prediction 1 to client.
Served encrypted prediction 2 to client.
Served encrypted prediction 3 to client.


In [ ]:
model.stop()
cluster.stop()

if not AUTO:
    process_ids = !ps aux | grep '[p]ython -m tf_encrypted.player --config' | awk '{print $2}'
    for process_id in process_ids:
        !kill {process_id}
        print("Process ID {id} has been killed.".format(id=process_id))